In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans as skKmeans

In [2]:
class KMeans():
    def __init__(self, n_clusters=8,
                 max_iter=300, tol=1e-4, random_state=0):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

    def _labels_inertia(self, X, centers):
        labels = np.zeros(X.shape[0])
        inertia = 0
        for sample_idx in range(X.shape[0]):
            min_dis = np.inf
            for center_idx in range(self.n_clusters):
                d = np.sum(np.square(X[sample_idx] - centers[center_idx]))
                if d < min_dis:
                    min_dis = d
                    labels[sample_idx] = center_idx
            inertia += min_dis
        return labels, inertia

    def fit(self, X):
        rng = np.random.RandomState(self.random_state)
        # consistent with scikit-learn
        tol = np.mean(np.var(X, axis=0)) * self.tol
        centers = X[rng.permutation(X.shape[0])[:self.n_clusters]]
        for i in range(self.max_iter):
            centers_old = centers.copy()
            labels, inertia = self._labels_inertia(X, centers)
            for center_idx in range(self.n_clusters):
                centers[center_idx] = np.mean(X[labels == center_idx], axis=0)
            center_shift_total = np.sum(np.square(centers_old - centers))
            if center_shift_total <= tol:
                break
        if center_shift_total > 0:
            labels, inertia = self._labels_inertia(X, centers)
        self.cluster_centers_ = centers
        self.labels_ = labels
        self.inertia_ = inertia
        self.n_iter_ = i + 1
        return self

    def predict(self, X):
        return self._labels_inertia(X, self.cluster_centers_)[0]

In [3]:
X, _ = load_iris(return_X_y=True)
clf1 = KMeans(n_clusters=3, random_state=0).fit(X)
clf2 = skKmeans(n_clusters=3, init="random", n_init=1, algorithm="full", random_state=0).fit(X)
assert np.allclose(clf1.cluster_centers_, clf2.cluster_centers_)
assert np.array_equal(clf1.labels_, clf2.labels_)
assert np.allclose(clf1.inertia_, clf2.inertia_)
pred1 = clf1.predict(X)
pred2 = clf2.predict(X)
assert np.array_equal(pred1, pred2)